# Day 12: GUI Development

## Learning Objectives:
- WebUI class
- Gradio interface
- File uploads
- Deployment strategies

### Prerequisites: Completed previous days
### Time: 1.5-2 hours

---

## Part 1: Introduction

Today we'll master GUI Development!

In [ ]:
# Setup imports
from qwen_agent.agents import Assistant
from qwen_agent.llm import get_chat_model
from qwen_agent.llm.schema import Message

print(f'Ready for {content["title"]}!')

## Part 2: Core Concepts

Key topics:
- WebUI class
- Gradio interface
- File uploads
- Deployment strategies


In [ ]:
# Example implementation
# See IMPLEMENTATION_PLAN.md for detailed examples

# Create basic configuration
llm_cfg = {'model': 'qwen-max-latest'}

# Initialize agent/tool
bot = Assistant(llm=llm_cfg)

# Test basic functionality
messages = [{'role': 'user', 'content': 'Hello!'}]
response = bot.run_nonstream(messages)
print(response[-1]['content'])

## Part 3: Practical Examples

Let's implement the concepts:

In [ ]:
# Example 1: Basic usage
# Refer to examples/ directory in repository
# See IMPLEMENTATION_PLAN.md for complete code

print('Example 1 complete')

In [ ]:
# Example 2: Advanced usage
# Check /examples directory for full implementations

print('Example 2 complete')

## Part 4: Practice Exercises

### Exercise 1: Create chatbot UI
### Exercise 2: Add custom styling
### Exercise 3: Deploy to cloud


In [ ]:
# TODO: Create chatbot UI
# Requirements:
# 1. Follow the pattern from examples
# 2. Test thoroughly
# 3. Handle errors

# Your code here:
# ...

## Key Takeaways

Today you learned:
- WebUI class
- Gradio interface
- File uploads
- Deployment strategies

### Common Patterns:
See IMPLEMENTATION_PLAN.md for detailed patterns.

### Next Steps:
Continue to Day 13!

### Resources:
- Qwen-Agent GitHub examples
- Official documentation
- IMPLEMENTATION_PLAN.md

🎉 Day 12 Complete!